## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-05-17-50-41 +0000,nypost,How to protect yourself against flesh-eating b...,https://nypost.com/2025/08/05/health/how-to-pr...
1,2025-08-05-17-49-33 +0000,nyt,What to Know About India’s Trade in Oil With R...,https://www.nytimes.com/2025/08/05/business/in...
2,2025-08-05-17-49-27 +0000,nyt,"Ion Iliescu, Who Steered Romania After Revolut...",https://www.nytimes.com/2025/08/05/world/europ...
3,2025-08-05-17-49-03 +0000,nyt,"Ozzy Osbourne Died of a Heart Attack, Death Ce...",https://www.nytimes.com/2025/08/05/arts/music/...
4,2025-08-05-17-48-15 +0000,nypost,Chuck Todd worries Democratic Party hasn’t ‘lo...,https://nypost.com/2025/08/05/media/chuck-todd...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2316/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
43,trump,50
67,house,17
324,new,16
124,gaza,15
174,up,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
238,2025-08-04-22-54-00 +0000,wsj,Democratic lawmakers blocked the Texas House f...,https://www.wsj.com/politics/policy/republican...,126
214,2025-08-05-01-00-00 +0000,wsj,President Trump’s high-stakes push to create m...,https://www.wsj.com/politics/texas-redistricti...,95
58,2025-08-05-16-22-40 +0000,nypost,"Trump walks along White House roof, jokes he’s...",https://nypost.com/2025/08/05/us-news/trump-wa...,88
237,2025-08-04-22-55-43 +0000,latimes,Trump plans White House task force on security...,https://www.latimes.com/politics/story/2025-08...,86
286,2025-08-04-20-30-20 +0000,nypost,Ex-Obama officials face federal grand jury pro...,https://nypost.com/2025/08/04/us-news/ex-obama...,84


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
238,126,2025-08-04-22-54-00 +0000,wsj,Democratic lawmakers blocked the Texas House f...,https://www.wsj.com/politics/policy/republican...
117,53,2025-08-05-13-27-00 +0000,cbc,"India, Russia hit back after U.S. accuses Modi...",https://www.cbc.ca/news/world/india-us-russia-...
258,52,2025-08-04-21-48-14 +0000,nypost,Brazil’s Supreme Court orders house arrest for...,https://nypost.com/2025/08/04/world-news/brazi...
272,45,2025-08-04-21-04-00 +0000,wsj,The State Department may require travelers ent...,https://www.wsj.com/politics/policy/trump-admi...
305,44,2025-08-04-18-44-19 +0000,nypost,Netanyahu approves plan to fully occupy Gaza S...,https://nypost.com/2025/08/04/world-news/netan...
115,44,2025-08-05-13-39-25 +0000,nypost,CEO ignored ‘red flags’ ahead of ‘preventable’...,https://nypost.com/2025/08/05/world-news/titan...
208,40,2025-08-05-01-17-08 +0000,latimes,Wealthy first-time candidate Cloobeck drops $1...,https://www.latimes.com/california/story/2025-...
51,35,2025-08-05-16-34-00 +0000,wsj,The Swiss president is heading to the U.S. in ...,https://www.wsj.com/economy/trade/swiss-presid...
303,31,2025-08-04-19-06-46 +0000,nypost,Far left Sen. Warren gushes over NYC mayoral c...,https://nypost.com/2025/08/04/us-news/far-left...
264,31,2025-08-04-21-23-51 +0000,latimes,Tesla awards Elon Musk $29 billion in stock am...,https://www.latimes.com/business/story/2025-08...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
